In [43]:
import pandas as pd
#import matplotlib.ticker as mtick
import numpy as np
from arcgis.gis import *
gis = GIS()

# Set working directory
os.chdir(r"E:\Github\K-12-Enrollment")
print("Now working in:", os.getcwd())

Now working in: E:\Github\K-12-Enrollment


In [44]:
#Layer name for K-12 enrollment data
lyrK12 = 'Schools_PreKto12'
lyrTAZ = 'USTM_TAZ_2021_09_22'

#Column Names
cnLevel    = 'SchoolLevel'
cnName     = 'SchoolName'
cnG00      = 'K'            #kindergarten as grade 0
cnG01      = 'Grade1'
cnG02      = 'Grade2'
cnG03      = 'Grade3'
cnG04      = 'Grade4'
cnG05      = 'Grade5'
cnG06      = 'Grade6'
cnG07      = 'Grade7'
cnG08      = 'Grade8'
cnG09      = 'Grade9'
cnG10      = 'Grade10'
cnG11      = 'Grade11'
cnG12      = 'Grade12'
cnTot      = 'TotalK12'
cnID       = 'SchoolID'
cnGroup    = 'Group'
cnPrivate  = 'PrivateSchool'
cnCharter  = 'CharterSchool'
cnOnlineS  = 'OnlineSchool'

#enrollment groups - school groups
egPrivateCharter = 'PriCha'
egPublic         = 'Public'
egTotal          = 'Enrol'

#enrollment levels - grade groups 
elElem = 'Elem'
elMidl = 'Midl'
elHigh = 'High'

#school levels - combo of original data and new levels
slElem = 'ELEM'
slMidl = 'MID'
slHigh = 'HIGH'
slKt12 = 'K12'
slKth8 = 'K8'
sl7t12 = '712'

#name of joined layers created later
lyrK12TAZ = 'Schools_PreKto12_withTAZ'
lyrTAZEnrol = 'USTM_TAZ_2021_09_22_Enrol'


In [45]:
#grade column names array
cnGrades   = [cnG00,cnG01,cnG02,cnG03,cnG04,cnG05,cnG06,cnG07,cnG08,cnG09,cnG10,cnG11,cnG12]

cnGroupEnrol = [egPublic+'_'+elElem,egPublic+'_'+elMidl,egPublic+'_'+elHigh,egPrivateCharter+'_'+elElem,egPrivateCharter+'_'+elMidl,egPrivateCharter+'_'+elHigh]
#display(cnGroupEnrol)

# list of Enrollment levels by Grade and by School_Level
colnames  =  [cnLevel, cnG00 , cnG01 , cnG02 , cnG03 , cnG04 , cnG05 , cnG06 , cnG07 , cnG08 , cnG09 , cnG10 , cnG11 , cnG12 ]
enroldata = [[slElem , elElem, elElem, elElem, elElem, elElem, elElem, elElem, elElem, elElem, elElem, elElem, elElem, elElem],
             [slMidl , elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl],
             [slHigh , elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh],
             [slKt12 , elElem, elElem, elElem, elElem, elElem, elElem, elElem, elMidl, elMidl, elMidl, elHigh, elHigh, elHigh],
             [slKth8 , elElem, elElem, elElem, elElem, elElem, elElem, elElem, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl],
             [sl7t12 , elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl, elHigh, elHigh, elHigh]
            ]
  
# Create the pandas DataFrame
dfEnrollLevels = pd.DataFrame(enroldata, columns = colnames)
print('\nEnrollment Levels by School Level and Grade:')
display(dfEnrollLevels)


Enrollment Levels by School Level and Grade:


,SchoolLevel,K,Grade1,Grade2,Grade3,Grade4,Grade5,Grade6,Grade7,Grade8,Grade9,Grade10,Grade11,Grade12
0,ELEM,Elem,Elem,Elem,Elem,Elem,Elem,Elem,Elem,Elem,Elem,Elem,Elem,Elem
1,MID,Midl,Midl,Midl,Midl,Midl,Midl,Midl,Midl,Midl,Midl,Midl,Midl,Midl
2,HIGH,High,High,High,High,High,High,High,High,High,High,High,High,High
3,K12,Elem,Elem,Elem,Elem,Elem,Elem,Elem,Midl,Midl,Midl,High,High,High
4,K8,Elem,Elem,Elem,Elem,Elem,Elem,Elem,Midl,Midl,Midl,Midl,Midl,Midl
5,712,Midl,Midl,Midl,Midl,Midl,Midl,Midl,Midl,Midl,Midl,High,High,High


In [46]:
dfK12 = pd.DataFrame.spatial.from_featureclass(lyrK12)

#display data totals before filtering
print ("Total Enrollment: " + "{:,}".format(dfK12[cnTot].sum()))
print ("Total Schools: "    + "{:,}".format(dfK12.shape[0])    )

#created filtered dataset that removes unnecessary rows

#make copy to preserve original data
dfK12_fltr = dfK12.copy()

#only school with enrollment
dfK12_fltr = dfK12_fltr[dfK12_fltr[cnTot] > 0]

#remove NONE and PREK schools, since no data
dfK12_fltr = dfK12_fltr[(dfK12_fltr[cnLevel] != 'PREK') &
                        (dfK12_fltr[cnLevel] != 'NONE')]

print ('Total Filtered Schools After NONE/PREK Removed: ' + "{:,}".format(dfK12_fltr.shape[0])
         + ' (' + str(round(dfK12_fltr.shape[0] / dfK12.shape[0] * 100,1)) + '%)')
print ("Total Filtered Enrollment After NONE/PREK Removed: " + "{:,}".format(dfK12_fltr[cnTot].sum()) + 
         ' (' + str(round(dfK12_fltr[cnTot].sum() / dfK12[cnTot].sum() * 100,1)) + '%)')

dfK12_fltr['OnlineSchool'] = dfK12_fltr['OnlineSchool'].fillna("")

#remove online schools
dfK12_fltr = dfK12_fltr[dfK12_fltr[cnOnlineS] != 'Y']
print ("Total Filtered Schools After Online Removed: " + "{:,}".format(dfK12_fltr.shape[0]) + 
         ' (' + str(round(dfK12_fltr.shape[0] / dfK12_fltr.shape[0] * 100,1)) + '%)')
print ("Total Filtered Enrollment After Online Removed: " + "{:,}".format(dfK12_fltr[cnTot].sum()) + 
         ' (' + str(round(dfK12_fltr[cnTot].sum() / dfK12[cnTot].sum() * 100,1)) + '%)')

Total Enrollment: 659,625
Total Schools: 1,377
Total Filtered Schools After NONE/PREK Removed: 1,004 (72.9%)
Total Filtered Enrollment After NONE/PREK Removed: 659,625 (100.0%)
Total Filtered Schools After Online Removed: 988 (100.0%)
Total Filtered Enrollment After Online Removed: 640,268 (97.1%)


In [47]:
#Set private and charter schools to 'PriCha' group
dfK12_fltr.loc[((dfK12_fltr[cnPrivate]=='True') |
                (dfK12_fltr[cnCharter]=='True')), cnGroup] = egPrivateCharter

#set non-private and non-charter schools to 'Public' group
dfK12_fltr.loc[((dfK12_fltr[cnPrivate]!='True') &
                (dfK12_fltr[cnCharter]!='True')), cnGroup] = egPublic

display(dfK12_fltr.groupby([cnGroup],as_index=False)
        .agg(NumSchools=(cnTot,'size'),NumStudents=(cnTot,'sum')))

#display(dfK12_filtered.groupby(['Group'],as_index=False).agg({'TotalK12':[np.size]})

,Group,NumSchools,NumStudents
0,PriCha,128,71485
1,Public,860,568783


In [48]:
#make copy to preserve pre reclassification
dfK12_fltr_rc = dfK12_fltr.copy()

#Explicitely reclassify K through 8, K through 12, and 7 through 12
dfK12_fltr_rc.loc[(((dfK12_fltr_rc[cnLevel]==slElem) & (dfK12_fltr_rc[cnG08]>0)) |
                    (dfK12_fltr_rc[cnLevel]==slMidl) & (dfK12_fltr_rc[cnG04]>0)), cnLevel] = slKth8
dfK12_fltr_rc.loc[ ((dfK12_fltr_rc[cnLevel]==slHigh) & (dfK12_fltr_rc[cnG00]>0)), cnLevel] = slKt12
dfK12_fltr_rc.loc[ ((dfK12_fltr_rc[cnLevel]==slHigh) & (dfK12_fltr_rc[cnG07]>0)), cnLevel] = sl7t12

#manual change for Inovations High School only
dfK12_fltr_rc.loc[(dfK12_fltr_rc[cnName]=='Innovations High School'), cnLevel] = slHigh

#display enrollment by grade to get understanding of the distribution of enrollment for each School_Level
print("\nPre-Reclassify:")
display(dfK12_fltr.groupby([cnLevel,cnGroup], as_index=False).agg(SchoolCount=(cnID,'size'),G00=(cnG00,'sum'),G01=(cnG01,'sum'),G02=(cnG02,'sum'),G03=(cnG03,'sum'),G04=(cnG04,'sum'),G05=(cnG05,'sum'),G06=(cnG06,'sum'),G07=(cnG07,'sum'),G08=(cnG08,'sum'),G09=(cnG09,'sum'),G10=(cnG10,'sum'),G11=(cnG11,'sum'),G12=(cnG12,'sum'),Tot=(cnTot,'sum')))

#display enrollment by grade to get understanding of the distribution of enrollment for each School_Level
print("\nPost-Reclassify:")
display(dfK12_fltr_rc.groupby([cnLevel,cnGroup], as_index=False).agg(SchoolCount=(cnID,'size'),G00=(cnG00,'sum'),G01=(cnG01,'sum'),G02=(cnG02,'sum'),G03=(cnG03,'sum'),G04=(cnG04,'sum'),G05=(cnG05,'sum'),G06=(cnG06,'sum'),G07=(cnG07,'sum'),G08=(cnG08,'sum'),G09=(cnG09,'sum'),G10=(cnG10,'sum'),G11=(cnG11,'sum'),G12=(cnG12,'sum'),Tot=(cnTot,'sum')))


Pre-Reclassify:


,SchoolLevel,Group,SchoolCount,G00,G01,G02,G03,G04,G05,G06,G07,G08,G09,G10,G11,G12,Tot
0,ELEM,PriCha,65,4549,4459,4500,4306,4242,3986,3635,1644,1361,286,49,40,25,33082
1,ELEM,Public,540,35407,36948,38936,41603,41897,41636,27534,97,86,0,3,1,2,264150
2,HIGH,PriCha,30,20,21,26,22,26,25,153,999,1024,1964,2359,2374,2109,11122
3,HIGH,Public,155,294,308,341,380,335,337,370,1186,1325,21622,49656,49614,48413,174181
4,K12,PriCha,14,935,943,934,962,966,986,1042,1184,1192,1016,875,737,664,12436
5,K12,Public,12,53,54,51,62,47,56,52,81,93,138,124,241,762,1814
6,MID,PriCha,19,1558,1546,1588,1531,1541,1685,1718,1428,1426,824,0,0,0,14845
7,MID,Public,153,14,11,13,19,15,642,14902,42093,44228,26698,3,0,0,128638



Post-Reclassify:


,SchoolLevel,Group,SchoolCount,G00,G01,G02,G03,G04,G05,G06,G07,G08,G09,G10,G11,G12,Tot
0,712,PriCha,14,0,0,0,0,0,0,126,970,970,1001,928,878,729,5602
1,712,Public,31,0,0,0,0,0,0,15,808,953,1603,2959,3044,2879,12261
2,ELEM,PriCha,35,2535,2528,2448,2418,2282,2147,1804,139,0,0,0,0,0,16301
3,ELEM,Public,535,35282,36840,38799,41494,41801,41506,27418,2,0,0,0,0,0,263142
4,HIGH,PriCha,15,0,0,0,0,0,0,0,0,41,963,1431,1496,1380,5311
5,HIGH,Public,123,0,0,0,1,0,1,2,0,15,19701,46419,46268,45231,157638
6,K12,PriCha,15,955,964,960,984,992,1011,1069,1213,1205,1016,875,737,664,12645
7,K12,Public,13,347,362,392,441,382,392,405,459,450,456,402,543,1065,6096
8,K8,PriCha,48,3572,3477,3640,3419,3501,3420,3445,2829,2684,1110,49,40,25,31211
9,K8,Public,6,139,119,149,128,111,141,116,95,86,0,3,1,2,1090


In [49]:
#investigate
#dfK12_fltr_rc[dfK12_fltr_rc[cnLevel]==sl7t12]

#blue peak high
#dfK12_fltr[dfK12_fltr[cnName]=='Blue Peak High']

#dfK12_fltr_rc[(dfK12_fltr_rc[cnLevel]==slElem) & (dfK12_fltr_rc[cnG12]>0)]

In [50]:
#calculate enrollment by level for all schools

#normalize (reverse-pivot) enrollment by grade for ease of calcs
dfK12_melt = pd.melt(dfK12_fltr_rc, id_vars=[cnID,cnLevel,cnGroup], value_vars=cnGrades)
dfK12_melt = dfK12_melt.rename(columns={'variable':'Grade','value':'Enrollment'})
#display(dfK12_melt)

#create 'lookup table' for Enrollment Level
dfEnrollLevels_melt = pd.melt(dfEnrollLevels, id_vars=[cnLevel], value_vars=cnGrades)
dfEnrollLevels_melt = dfEnrollLevels_melt.rename(columns={'variable':'Grade','value':'EnrollmentLevel'})
#display(dfEnrollLevels_melt)

#join to get enrollment levels
dfK12_enrolllevels = pd.DataFrame.merge(dfK12_melt,dfEnrollLevels_melt,on=(cnLevel,'Grade'))

#group to get subtotals by School, Group, and Enrollment Level
dfK12_enrolllevels = dfK12_enrolllevels.groupby([cnID,cnGroup,'EnrollmentLevel'],as_index=False).agg(Enrollment=('Enrollment','sum'))
dfK12_enrolllevels

,SchoolID,Group,EnrollmentLevel,Enrollment
0,132,Public,Elem,535
1,133,Public,Elem,416
2,134,Public,Elem,531
3,135,Public,Elem,509
4,136,Public,Elem,876
...,...,...,...,...
1137,186822,PriCha,Midl,42
1138,186823,Public,Elem,544
1139,186836,Public,Elem,144
1140,186836,Public,Midl,17


In [51]:

dfK12_enrol = dfK12_enrolllevels.pivot(index=[cnID], columns=[cnGroup,'EnrollmentLevel'],values='Enrollment')

#collapse multi-index and include '_' between index values
dfK12_enrol.columns = ['_'.join(col).strip() for col in dfK12_enrol.columns.values]

dfK12_enrol = dfK12_enrol.fillna(0)
dfK12_enrol = dfK12_enrol.astype(int)

dfK12_enrol = dfK12_enrol[cnGroupEnrol]
display(dfK12_enrol)

display(dfK12_enrol.sum())
display(dfK12_enrol.sum().sum())

,Public_Elem,Public_Midl,Public_High,PriCha_Elem,PriCha_Midl,PriCha_High
SchoolID,,,,,,
132,535,0,0,0,0,0
133,416,0,0,0,0,0
134,531,0,0,0,0,0
135,509,0,0,0,0,0
136,876,0,0,0,0,0
...,...,...,...,...,...,...
186816,0,0,0,521,259,0
186822,0,0,0,167,42,0
186823,544,0,0,0,0,0


Public_Elem    266766
Public_Midl    133487
Public_High    168530
PriCha_Elem     47710
PriCha_Midl     13653
PriCha_High     10122
dtype: int64

640268

In [53]:
#export school enrollment results to csv
strK12File = "results\k12_enrol.csv"

dfK12_enrol.to_csv(strK12File)

#join csv with feature class
arcpy.management.AddJoin(lyrK12, cnID, strK12File, cnID,'KEEP_ALL')

#spatial join to TAZ layers
arcpy.analysis.SpatialJoin(lyrK12, lyrTAZ, lyrK12TAZ, join_operation='JOIN_ONE_TO_ONE',join_type='KEEP_COMMON',match_option='WITHIN')

#remove join
arcpy.RemoveJoin_management(lyrK12)

<Result 'Schools_PreKto12'>

In [54]:
#get joined school/TAZ layer in order to aggregate to CO_TAZID
sdfK12TAZ = pd.DataFrame.spatial.from_featureclass(lyrK12TAZ)
dfK12TAZ = sdfK12TAZ[['SchoolName','CO_TAZID'] + cnGroupEnrol]
dfK12TAZ = dfK12TAZ.fillna(0)
#display(dfK12TAZ.columns)
display(dfK12TAZ)

,SchoolName,CO_TAZID,Public_Elem,Public_Midl,Public_High,PriCha_Elem,PriCha_Midl,PriCha_High
0,River Rock School,490345,932,0,0,0,0,0
1,Skyridge High School,490404,0,114,2500,0,0,0
2,Springside School,490284,821,0,0,0,0,0
3,Summit High,490663,0,19,56,0,0,0
4,American Academy of Innovation,351062,0,0,0,0,200,174
...,...,...,...,...,...,...,...,...
1372,Square Peg Academy,490791,0,0,0,0,0,0
1373,Stokes Nature Preschool,50882,0,0,0,0,0,0
1374,Mountain Point Academy,490397,0,0,0,0,0,0
1375,KoolMinds Orem,490764,0,0,0,0,0,0


In [57]:
#aggregate by CO_TAZID
dfTAZSummary = dfK12TAZ.groupby(['CO_TAZID']).agg(sum)
dfTAZSummary[dfTAZSummary.select_dtypes(include='number').columns] = \
    dfTAZSummary.select_dtypes(include='number').astype(int)

#remove rows with all zeros
dfTAZSummary = dfTAZSummary.loc[~(dfTAZSummary==0).all(axis=1)]

dfTAZSummary

,SchoolName,Public_Elem,Public_Midl,Public_High,PriCha_Elem,PriCha_Midl,PriCha_High
CO_TAZID,,,,,,,
1007,Beaver Preschool,0,0,0,0,0,0
1009,Belknap School,451,0,0,0,0,0
1011,Beaver High,0,225,221,0,0,0
1062,Milford High,0,95,111,0,0,0
1064,Milford School,211,0,0,0,0,0
...,...,...,...,...,...,...,...
570408,Burch Creek School,579,0,0,0,0,0
570412,H Guy Child SchoolSouth Ogden Jr High,380,801,0,0,0,0
570416,Providence Montessori Academy,0,0,0,0,0,0


In [58]:
#calculate total enrollment for each level by adding Private/Charter and Public schools
dfTAZSummary[egTotal+'_'+elElem]  = dfTAZSummary[egPublic+'_'+elElem] + dfTAZSummary[egPrivateCharter+'_'+elElem]
dfTAZSummary[egTotal+'_'+elMidl]  = dfTAZSummary[egPublic+'_'+elMidl] + dfTAZSummary[egPrivateCharter+'_'+elMidl]
dfTAZSummary[egTotal+'_'+elHigh]  = dfTAZSummary[egPublic+'_'+elHigh] + dfTAZSummary[egPrivateCharter+'_'+elHigh]
dfTAZSummary

,SchoolName,Public_Elem,Public_Midl,Public_High,PriCha_Elem,PriCha_Midl,PriCha_High,Enrol_Elem,Enrol_Midl,Enrol_High
CO_TAZID,,,,,,,,,,
1007,Beaver Preschool,0,0,0,0,0,0,0,0,0
1009,Belknap School,451,0,0,0,0,0,451,0,0
1011,Beaver High,0,225,221,0,0,0,0,225,221
1062,Milford High,0,95,111,0,0,0,0,95,111
1064,Milford School,211,0,0,0,0,0,211,0,0
...,...,...,...,...,...,...,...,...,...,...
570408,Burch Creek School,579,0,0,0,0,0,579,0,0
570412,H Guy Child SchoolSouth Ogden Jr High,380,801,0,0,0,0,380,801,0
570416,Providence Montessori Academy,0,0,0,0,0,0,0,0,0


In [61]:
#check the final data

#subtotals by enrollment group and level
display(dfTAZSummary.select_dtypes(include='number').sum())

#subtotals by enrollment group
print (egPublic        ,':',dfTAZSummary[[egPublic        +'_'+elElem,egPublic        +'_'+elMidl,egPublic        +'_'+elHigh]].sum().sum())
print (egPrivateCharter,':',dfTAZSummary[[egPrivateCharter+'_'+elElem,egPrivateCharter+'_'+elMidl,egPrivateCharter+'_'+elHigh]].sum().sum())
print (egTotal         ,':',dfTAZSummary[[egTotal         +'_'+elElem,egTotal         +'_'+elMidl,egTotal         +'_'+elHigh]].sum().sum())

Public_Elem    266994
Public_Midl    133506
Public_High    168554
PriCha_Elem     47710
PriCha_Midl     13653
PriCha_High     10122
Enrol_Elem     314704
Enrol_Midl     147159
Enrol_High     178676
dtype: int64

Public : 569054
PriCha : 71485
Enrol : 640539


In [62]:
#export CO_TAZID enrollment to csv

sFilename = 'results\\K12_Enrollment_' + lyrTAZ + '.csv'

dfTAZSummary.to_csv(sFilename)
print('CSV Exported to: ' + sFilename)

CSV Exported to: results\K12_Enrollment_USTM_TAZ_2021_09_22.csv


In [63]:
#spatial join to TAZ layer
arcpy.management.AddJoin(lyrTAZ,'CO_TAZID','results\\K12_Enrollment_' + lyrTAZ + '.csv','CO_TAZID','KEEP_ALL')

#perform code while joined
arcpy.management.CopyFeatures(lyrTAZ, 'results\\taz_with_enrollment.shp')

#remove join so that code can be rerun
arcpy.RemoveJoin_management(lyrTAZ)

<Result 'USTM_TAZ_2021_09_22'>